In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np

from sklearn.utils import resample
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import keras

plt.style.use('ggplot')

Using TensorFlow backend.
C:\Users\chait\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\chait\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\chait\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\chait\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
# Initial data preparation

def add_derived_columns(df):
    df_ext = df.copy()
    df_ext['jid'] = df_ext['uid'].map(str) + '_' + df_ext['conversion_id'].map(str)
    
    min_max_scaler = MinMaxScaler()
    for cname in ('timestamp', 'time_since_last_click'):
        x = df_ext[cname].values.reshape(-1, 1) 
        df_ext[cname + '_norm'] = min_max_scaler.fit_transform(x)
    
    return df_ext

def filter_journeys_by_length(df, min_touchpoints):
    if min_touchpoints <= 1:
        return df
    else:
        grouped = df.groupby(['jid'])['uid'].count().reset_index(name="count")
        return df[df['jid'].isin( grouped[grouped['count'] >= min_touchpoints]['jid'].values )]

def sample_campaigns(df, n_campaigns):    
    campaigns = np.random.choice( df['campaign'].unique(), n_campaigns, replace = False )
    return df[ df['campaign'].isin(campaigns) ]

def balance_conversions(df):
    df_minority = df[df.conversion == 1]
    df_majority = df[df.conversion == 0]
    
    df_majority_jids = np.array_split(df_majority['jid'].unique(), 100 * df_majority.shape[0]/df_minority.shape[0] )
    
    df_majority_sampled = pd.DataFrame(data=None, columns=df.columns)
    for jid_chunk in df_majority_jids:
        df_majority_sampled = pd.concat([df_majority_sampled, df_majority[df_majority.jid.isin(jid_chunk)]])
        if df_majority_sampled.shape[0] > df_minority.shape[0]:
            break
    
    return pd.concat([df_majority_sampled, df_minority]).sample(frac=1).reset_index(drop=True)

def map_one_hot(df, column_names, result_column_name):
    mapper = {} 
    for i, col_name in enumerate(column_names):
        for val in df[col_name].unique():
            mapper[str(val) + str(i)] = len(mapper)
         
    df_ext = df.copy()
    
    def one_hot(values):
        v = np.zeros( len(mapper) )
        for i, val in enumerate(values): 
            v[ mapper[str(val) + str(i)] ] = 1
        return v    
    
    df_ext[result_column_name] = df_ext[column_names].values.tolist()
    df_ext[result_column_name] = df_ext[result_column_name].map(one_hot)
    
    return df_ext
    
data_file = 'criteo_attribution_dataset.tsv.gz'
df0 = pd.read_csv(data_file, sep='\t', compression='gzip')

n_campaigns = 50

df1 = add_derived_columns(df0)
df2 = sample_campaigns(df1, n_campaigns)
df3 = filter_journeys_by_length(df2, 2)
df4 = balance_conversions(df3)
df5 = map_one_hot(df4, ['cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat8'], 'cats')
df6 = map_one_hot(df5, ['campaign'], 'campaigns').sort_values(by=['timestamp_norm'])

print(df6.shape[0])
print([df6[df6.conversion == 0].shape[0], df6[df6.conversion == 1].shape[0]])

C:\Users\chait\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\chait\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


107792
[54193, 53599]


# First Touch Attribution

In [3]:
def  first_touch_attribution(df):
    
    def count_by_campaign(df):
        counters = np.zeros(n_campaigns)
        for campaign_one_hot in df['campaigns'].values:
            campaign_id = np.argmax(campaign_one_hot)
            counters[campaign_id] = counters[campaign_id] + 1
        return counters
        
    campaign_impressions = count_by_campaign(df)
    
    df_converted = df[df['conversion'] == 1]
    idx = df_converted.groupby(['jid'])['timestamp_norm'].transform(min) == df_converted['timestamp_norm']
    campaign_conversions = count_by_campaign(df_converted[idx])
    print(campaign_conversions / campaign_impressions)   
    return campaign_conversions / campaign_impressions

# Linear Attribution

In [4]:
def linear_touch_attribution(df):
    
    def count_by_campaign(df):
        counters = np.zeros(n_campaigns)
        for campaign_one_hot in df['campaigns'].values:
            campaign_id = np.argmax(campaign_one_hot)
            counters[campaign_id] = counters[campaign_id] + 1
        return counters
        
    campaign_impressions = count_by_campaign(df)
    
    df_converted = df[df['conversion'] == 1]
#    idx = df_converted.groupby(['jid'])['timestamp_norm'].transform(min) == df_converted['timestamp_norm']
    campaign_conversions = count_by_campaign(df_converted)
        
    return campaign_conversions / campaign_impressions
    

# U Shaped Attribution

In [ ]:
def U_touch_attribution(df):
    def count_by_campaign(df):
        counters = np.zeros(n_campaigns)
        for campaign_one_hot in df['campaigns'].values:
            campaign_id = np.argmax(campaign_one_hot)
            counters[campaign_id] = counters[campaign_id] + 1
        return counters
    campaign_impressions = count_by_campaign(df)
    df_converted = df[df['conversion'] == 1]
    temp_jid=0
    campaign_conversions_nrml=0
    campaign_conversions_first=0
    campaign_conversions_last=0
    campaign_conversions_intermediate=0
    for jid in df_converted.jid.unique():
        if jid != temp_jid:
            temp_jid = jid
            if df_converted[df_converted['jid'] == temp_jid]['timestamp_norm'].max() == df_converted[df_converted['jid'] == temp_jid]['timestamp_norm'].min():
                idx_nrml = df_converted.groupby(['jid'])['timestamp_norm'].transform(min) == df_converted['timestamp_norm']
                campaign_conversions_nrml = count_by_campaign(df_converted[idx_nrml])
            if df_converted[df_converted['jid'] == temp_jid]['timestamp_norm'].max() != df_converted[df_converted['jid'] == temp_jid]['timestamp_norm'].min():
                idx_min = df_converted.groupby(['jid'])['timestamp_norm'].transform(min) == df_converted['timestamp_norm']
                idx_max = df_converted.groupby(['jid'])['timestamp_norm'].transform(max) == df_converted['timestamp_norm']
                campaign_conversions_first = count_by_campaign(df_converted[idx_min])
                campaign_conversions_last = count_by_campaign(df_converted[idx_max])
                campaign_conversions_intermediate = (campaign_impressions - (campaign_conversions_first + campaign_conversions_last ))
    return (((campaign_conversions_nrml + campaign_conversions_first + campaign_conversions_last ) * .7) + (campaign_conversions_intermediate * .3)) / campaign_impressions
uta = U_touch_attribution(df6)

In [40]:
dummy = df6
dummy_mod=dummy[dummy['conversion']==1]
dummy_mod.campaign.nunique()

50

In [41]:
def U_shape_attribution(dummy):
    dummy = df6
    dummy_mod.sort_values(by=['campaign','timestamp'], inplace=True)
    temp_jid = 0 
    dummy_array=[]
    campaign_array=[]
    timestamp_array=[]
    ctr=0
    for campaign in dummy_mod.campaign.unique():
             ctr = ctr + 1
             print("Campaign No:", ctr,campaign )
             for jid in  dummy_mod[dummy_mod['campaign']==campaign].jid.unique():
                if jid != temp_jid:
                    count = 1
                    temp_jid = jid
                    for idx,timestamp in enumerate (dummy_mod[dummy_mod['jid']==jid]['timestamp']):
                        timestamp_array.append(idx+1)
                        campaign_array.append(campaign)
                        if timestamp == dummy_mod[dummy_mod['jid']==jid]['timestamp'].max():
                            dummy_array.append(100)
                        elif timestamp == dummy_mod[dummy_mod['jid']==jid]['timestamp'].min():
                            dummy_array.append(100)
                        elif timestamp < dummy_mod[dummy_mod['jid']==jid]['timestamp'].max():
                            dummy_array.append(50)
                else:
                     count = count + 1

C:\Users\chait\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Campaign No: 1 73322
Campaign No: 2 2869134
Campaign No: 3 2946551
Campaign No: 4 3139444
Campaign No: 5 3610770
Campaign No: 6 4800911
Campaign No: 7 4814007
Campaign No: 8 4828354
Campaign No: 9 4869923
Campaign No: 10 5743167
Campaign No: 11 7336375
Campaign No: 12 7828339
Campaign No: 13 7892338
Campaign No: 14 8287704
Campaign No: 15 8403848
Campaign No: 16 9106407
Campaign No: 17 9700342
Campaign No: 18 10013589
Campaign No: 19 10383101
Campaign No: 20 11289564
Campaign No: 21 11765782
Campaign No: 22 13445189
Campaign No: 23 13576416
Campaign No: 24 14121532
Campaign No: 25 14576401
Campaign No: 26 15073876
Campaign No: 27 15574227
Campaign No: 28 16040127
Campaign No: 29 16184517
Campaign No: 30 17661709
Campaign No: 31 17686795
Campaign No: 32 18443077
Campaign No: 33 18836888
Campaign No: 34 20717215
Campaign No: 35 21443097
Campaign No: 36 24316983
Campaign No: 37 25998434
Campaign No: 38 26091099
Campaign No: 39 26903589
Campaign No: 40 26945970
Campaign No: 41 27777499
Cam

In [42]:
dummy[dummy['jid'] == 7306395_3063962]['timestamp']

Series([], Name: timestamp, dtype: object)

In [43]:
dummy_mod.campaign.unique()

array([73322, 2869134, 2946551, 3139444, 3610770, 4800911, 4814007,
       4828354, 4869923, 5743167, 7336375, 7828339, 7892338, 8287704,
       8403848, 9106407, 9700342, 10013589, 10383101, 11289564, 11765782,
       13445189, 13576416, 14121532, 14576401, 15073876, 15574227,
       16040127, 16184517, 17661709, 17686795, 18443077, 18836888,
       20717215, 21443097, 24316983, 25998434, 26091099, 26903589,
       26945970, 27777499, 27888153, 28389117, 28752986, 29091102,
       29513915, 29531976, 30368132, 30534043, 32368244], dtype=object)

In [47]:
dummy = df6[:3]

U_shape_attribution(dummy)

C:\Users\chait\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide


In [45]:
leaderboard = pd.DataFrame({'Timestamp':timestamp_array,'Campaign':campaign_array,'Wightage':dummy_array})
leaderboard

,Timestamp,Campaign,Wightage
0,1,73322,100
1,2,73322,50
2,3,73322,100
3,1,73322,100
4,2,73322,100
...,...,...,...
53594,2,32368244,100
53595,1,32368244,100
53596,2,32368244,100
53597,1,32368244,100


# Time Decay Attribution

In [56]:
def timedecay_attribution(df):
    def count_by_campaign(df):
        counters = np.zeros(n_campaigns)
        for campaign_one_hot in df['campaigns'].values:
            campaign_id = np.argmax(campaign_one_hot)
            counters[campaign_id] = counters[campaign_id] + 1
        return counters
    campaign_impressions = count_by_campaign(df)
    df_converted = df[df['conversion'] == 1]
    temp_jid=0
    campaign_conversions_nrml=0
    campaign_conversions_first=0
    campaign_conversions_last=0
    campaign_conversions_intermediate=0
    for jid in df_converted.jid.unique():
        if jid != temp_jid:
            temp_jid = jid
            if df_converted[df_converted['jid'] == temp_jid]['timestamp_norm'].max() == df_converted[df_converted['jid'] == temp_jid]['timestamp_norm'].min():
                idx_nrml = df_converted.groupby(['jid'])['timestamp_norm'].transform(min) == df_converted['timestamp_norm']
                campaign_conversions_nrml = count_by_campaign(df_converted[idx_nrml])
            if df_converted[df_converted['jid'] == temp_jid]['timestamp_norm'].max() != df_converted[df_converted['jid'] == temp_jid]['timestamp_norm'].min():
                idx_min = df_converted.groupby(['jid'])['timestamp_norm'].transform(min) == df_converted['timestamp_norm']
                idx_max = df_converted.groupby(['jid'])['timestamp_norm'].transform(max) == df_converted['timestamp_norm']
                campaign_conversions_first = count_by_campaign(df_converted[idx_min])
                campaign_conversions_last = count_by_campaign(df_converted[idx_max])
                campaign_conversions_intermediate = (campaign_impressions - (campaign_conversions_first + campaign_conversions_last ))
    return (campaign_conversions_nrml + (campaign_conversions_first * 0.1) + (campaign_conversions_last  * 0.6) + (campaign_conversions_intermediate * 0.3)) / campaign_impressions
tda = timedecay_attribution(df6)

2380977_28647208
Multiple Records 
8361406_13161205
Multiple Records 
20280968_16828844
Multiple Records 
12324598_3958887
Multiple Records 
8361406_32142633
Multiple Records 
15366984_8769444
Multiple Records 
26197824_19633474
Multiple Records 
2971919_22818643
Multiple Records 
5039198_8467149
Multiple Records 
1502462_1446335
Multiple Records 
11090808_29891747
Multiple Records 
25519218_29930234
Multiple Records 
13791416_10482840
Multiple Records 
12717203_19092620
Multiple Records 
2013817_15081096
Multiple Records 
12878239_13360384
Multiple Records 
20222491_4934012
Multiple Records 
17917877_813345
Multiple Records 
5880485_22216717
Multiple Records 
12764645_7599929
Multiple Records 
22664906_12638681
Multiple Records 
13866458_14203634
Multiple Records 
12764645_8588436
Multiple Records 
21218387_20352785
Multiple Records 
3687371_9190558
Multiple Records 
10944629_2897014
Multiple Records 
16716650_18522647
Multiple Records 
24503546_9017326
Multiple Records 
28120722_3062

12050960_13438055
Multiple Records 
24304961_30582171
Multiple Records 
18772682_20103884
Multiple Records 
14142903_8913502
Multiple Records 
19066263_17557019
Multiple Records 
11339117_4462530
Multiple Records 
7872880_9665134
Multiple Records 
3468362_10578437
Multiple Records 
26363717_32291123
Multiple Records 
10592204_29332094
Multiple Records 
25365907_29137276
Multiple Records 
2740515_14909521
Multiple Records 
12452393_5523784
Multiple Records 
4024116_28419006
Multiple Records 
6621605_17207266
Multiple Records 
4105152_7340521
Multiple Records 
12221735_1858946
Multiple Records 
14007231_10384608
Multiple Records 
4710308_15016631
Multiple Records 
25532789_5635322
Multiple Records 
4253518_594881
Multiple Records 
13680204_21111611
Multiple Records 
29987943_25305346
Multiple Records 
4269938_25577914
Multiple Records 
9894165_27190634
Multiple Records 
28080437_21754430
Multiple Records 
5329464_2015449
Multiple Records 
23665524_9397253
Multiple Records 
13329442_13282

23691211_9403469
Multiple Records 
20722866_2605587
Multiple Records 
11733976_21379925
Multiple Records 
20728332_21620171
Multiple Records 
19485432_25801529
Multiple Records 
18418258_1661421
Multiple Records 
25752968_28087688
Multiple Records 
13322530_14497177
Multiple Records 
19075807_23369097
Multiple Records 
28096469_30565578
Multiple Records 
16922099_28703920
Multiple Records 
8958546_14384371
Multiple Records 
22238900_25352806
Multiple Records 
656_26044415
Multiple Records 
9896103_26194642
Multiple Records 
31614689_2071962
Multiple Records 
25455483_31200158
Multiple Records 
26579715_15840198
Multiple Records 
10682339_18098694
Multiple Records 
17266155_16808146
Multiple Records 
987848_6298062
Multiple Records 
24524866_15351125
Multiple Records 
19996368_28722126
Multiple Records 
22315321_3411529
Multiple Records 
16034825_27144837
Multiple Records 
26579715_13541272
Multiple Records 
8217523_3950909
Multiple Records 
28711415_21990602
Multiple Records 
13466863_

12068331_18900301
Multiple Records 
7635031_23756043
Multiple Records 
25371769_25710831
Multiple Records 
19868917_30411818
Multiple Records 
21463619_7582220
Multiple Records 
24334051_7797374
Multiple Records 
1818520_1683944
Multiple Records 
23101725_18091645
Multiple Records 
22739271_24231736
Multiple Records 
8493391_666063
Multiple Records 
4769138_21807096
Multiple Records 
19014454_16457049
Multiple Records 
1369511_22513017
Multiple Records 
31560904_15232277
Multiple Records 
30381795_31372084
Multiple Records 
4166732_32046711
Multiple Records 
10925354_9099093
Multiple Records 
7567128_16159174
Multiple Records 
3332575_29207552
Multiple Records 
27545340_16563017
Multiple Records 
5871610_536333
Multiple Records 
31574921_10365235
Multiple Records 
27520667_1472692
Multiple Records 
22598868_29637229
Multiple Records 
7172339_20477717
Multiple Records 
25240740_10798123
Multiple Records 
13008349_24799546
Multiple Records 
4555870_20001923
Multiple Records 
21409154_231

31007318_8971016
Multiple Records 
11322742_20943015
Multiple Records 
30629142_31990453
Multiple Records 
11174296_14111918
Multiple Records 
7845033_24461897
Multiple Records 
9200193_30042417
Multiple Records 
18855826_22546408
Multiple Records 
6347416_3556613
Multiple Records 
23750213_29180015
Multiple Records 
1604231_18536875
Multiple Records 
16928432_26911063
Multiple Records 
9103877_3504390
Multiple Records 
26765453_30730275
Multiple Records 
14466004_10560923
Multiple Records 
27359326_30261660
Multiple Records 
2892558_23370820
Multiple Records 
17925013_14862930
Multiple Records 
26278245_26186373
Multiple Records 
17110315_14818411
Multiple Records 
6347416_18778584
Multiple Records 
27157669_9191224
Multiple Records 
30981592_24319573
Multiple Records 
27067739_23416982
Multiple Records 
12876297_22178431
Multiple Records 
13955253_22062268
Multiple Records 
10074449_6457906
Multiple Records 
30554498_23908353
Multiple Records 
15721716_30534877
Multiple Records 
2286

...

25725766_24423132
Multiple Records 
13757659_25328177
Multiple Records 
15892786_22825458
Multiple Records 
27221868_5465014
Multiple Records 
27882069_24415813
Multiple Records 
27427887_14129208
Multiple Records 
2920795_28912618
Multiple Records 
17951382_28672061
Multiple Records 
23129787_3300412
Multiple Records 
32103049_9969134
Multiple Records 
6844697_18199918
Multiple Records 
19125998_22621034
Multiple Records 
27374725_16140973
Multiple Records 
5924402_26137559
Multiple Records 
15897706_3586094
Multiple Records 
29572432_11221715
Multiple Records 
27161414_4400577
Multiple Records 
28693190_5360663
Multiple Records 
29696757_22433656
Multiple Records 
22399911_32207196
Multiple Records 
19318275_22486126
Multiple Records 
28886304_3650810
Multiple Records 
2174796_25731571
Multiple Records 
5827565_14631335
Multiple Records 
26661689_3904650
Multiple Records 
21794574_1230635
Multiple Records 
24436869_26794431
Multiple Records 
11778862_473712
Multiple Records 
21931470

22532413_22093104
Multiple Records 
30905563_17428814
Multiple Records 
29696405_1626986
Multiple Records 
4466455_29018276
Multiple Records 
13095212_29091980
Multiple Records 
30456416_25652136
Multiple Records 
26657401_5449201
Multiple Records 
800869_4703247
Multiple Records 
21760812_12939975
Multiple Records 
20279972_28930551
Multiple Records 
30789311_10290381
Multiple Records 
10691049_31086430
Multiple Records 
3009472_29276103
Multiple Records 
27210224_20722414
Multiple Records 
21338192_32442565
Multiple Records 
26232013_20684176
Multiple Records 
17161429_26563217
Multiple Records 
17677451_30094704
Multiple Records 
17677451_2403093
Multiple Records 
23513845_12162236
Multiple Records 
22586299_7781374
Multiple Records 
27187987_23149781
Multiple Records 
26453371_21773237
Multiple Records 
4679993_29082937
Multiple Records 
2335064_13909091
Multiple Records 
17406499_6932070
Multiple Records 
1485794_6053998
Multiple Records 
7965339_5156257
Multiple Records 
30591187

2089992_1109427
Multiple Records 
10499457_19366662
Multiple Records 
22138835_3080231
Multiple Records 
24907644_5073089
Multiple Records 
27840246_17089557
Multiple Records 
4533674_13759004
Multiple Records 
7650799_17911594
Multiple Records 
858879_23939160
Multiple Records 
5551493_3952051
Multiple Records 
700293_13801395
Multiple Records 
7091983_2155885
Multiple Records 
23621028_19712425
Multiple Records 
23587433_17655697
Multiple Records 
8999298_28003601
Multiple Records 
9200193_58676
Multiple Records 
31181559_14407297
Multiple Records 
23273031_149003
Multiple Records 
26773969_9032002
Multiple Records 
8063826_10183420
Multiple Records 
30804314_14631822
Multiple Records 
24407427_19247645
Multiple Records 
23124251_25927186
Multiple Records 
12520618_22439169
Multiple Records 
21003672_8252045
Multiple Records 
5195410_19096178
Multiple Records 
14138186_26010016
Multiple Records 
18575204_29657700
Multiple Records 
11998875_28244662
Multiple Records 
19994716_1899818


1286380_21558967
Multiple Records 
26051305_30146373
Multiple Records 
10344574_7373474
Multiple Records 
15882446_3259396
Multiple Records 
10130533_6277124
Multiple Records 
21992126_22817853
Multiple Records 
31132505_10307929
Multiple Records 
30095405_20743775
Multiple Records 
30112680_307966
Multiple Records 
24702964_25275440
Multiple Records 
19368585_8805203
Multiple Records 
12432238_4032347
Multiple Records 
26470151_1304407
Multiple Records 
7940338_14527675
Multiple Records 
6065492_29194074
Multiple Records 
14366622_26740243
Multiple Records 
19256942_25776131
Multiple Records 
32243887_30906287
Multiple Records 
21675430_14718193
Multiple Records 
10622419_32430881
Multiple Records 
24280317_3020386
Multiple Records 
30985750_7576487
Multiple Records 
16511078_10408966
Multiple Records 
5793356_18097992
Multiple Records 
32028938_17043918
Multiple Records 
1042500_3306912
Multiple Records 


KeyboardInterrupt: 

# Logistic Regression

In [13]:
def features_for_logistic_regression(df):

    def pairwise_max(series):
        return np.max(series.tolist(), axis = 0).tolist()
    
    aggregation = {
        'campaigns': pairwise_max,
        'cats': pairwise_max,
        'click': 'sum',
        'cost': 'sum',
        'conversion': 'max'
    }
    
    df_agg = df.groupby(['jid']).agg(aggregation)
    
    df_agg['features'] = df_agg[['campaigns', 'cats', 'click', 'cost']].values.tolist()
    
    return (
        np.stack(df_agg['features'].map(lambda x: np.hstack(x)).values),
        df_agg['conversion'].values
    )

In [14]:
x, y = features_for_logistic_regression(df6)
print(np.shape(x))

(26739, 301)


In [15]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.20, random_state = 1)

In [16]:

# Quick sanity check
from sklearn.linear_model import LogisticRegression

logisticRegr = LogisticRegression()
logisticRegr.fit(x_train, y_train)
score = logisticRegr.score(x_test, y_test)
print(score)

C:\Users\chait\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8264771877337322


In [17]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.constraints import NonNeg

m = np.shape(x)[1]
    
model = Sequential()  
model.add(Dense(1, input_dim=m, activation='sigmoid', name = 'contributions')) 

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy']) 
history = model.fit(x_train, y_train, batch_size=128, epochs=10, verbose=1, validation_data=(x_val, y_val)) 
score = model.evaluate(x_test, y_test, verbose=0) 
print('Test score:', score[0]) 
print('Test accuracy:', score[1])

Train on 17112 samples, validate on 4279 samples
Epoch 1/10
17112/17112 [==============================] - 0s 21us/step - loss: 0.6323 - acc: 0.6412 - val_loss: 0.5716 - val_acc: 0.7548
Epoch 2/10
17112/17112 [==============================] - 0s 12us/step - loss: 0.5379 - acc: 0.7663 - val_loss: 0.5155 - val_acc: 0.7773
Epoch 3/10
17112/17112 [==============================] - 0s 13us/step - loss: 0.4973 - acc: 0.7801 - val_loss: 0.4884 - val_acc: 0.7817
Epoch 4/10
17112/17112 [==============================] - 0s 13us/step - loss: 0.4765 - acc: 0.7874 - val_loss: 0.4740 - val_acc: 0.7908
Epoch 5/10
17112/17112 [==============================] - 0s 13us/step - loss: 0.4638 - acc: 0.7942 - val_loss: 0.4649 - val_acc: 0.7962
Epoch 6/10
17112/17112 [==============================] - 0s 14us/step - loss: 0.4552 - acc: 0.7995 - val_loss: 0.4579 - val_acc: 0.7978
Epoch 7/10
17112/17112 [==============================] - 0s 13us/step - loss: 0.4484 - acc: 0.8017 - val_loss: 0.4528 - val_acc:

# Last Touch Attribution

In [18]:
def last_touch_attribution(df):
    
    def count_by_campaign(df):
        counters = np.zeros(n_campaigns)
        for campaign_one_hot in df['campaigns'].values:
            campaign_id = np.argmax(campaign_one_hot)
            counters[campaign_id] = counters[campaign_id] + 1
        return counters
        
    campaign_impressions = count_by_campaign(df)
    
    df_converted = df[df['conversion'] == 1]
    idx = df_converted.groupby(['jid'])['timestamp_norm'].transform(max) == df_converted['timestamp_norm']
    campaign_conversions = count_by_campaign(df_converted[idx])
    #print(campaign_conversions / campaign_impressions) 
    return campaign_conversions / campaign_impressions

In [20]:
lta = last_touch_attribution(df6)
fta = first_touch_attribution(df6)
la = linear_touch_attribution(df6)

[0.204914   0.2132775  0.06685769 0.27150838 0.01953908 0.15900755
 0.17202572 0.22299441 0.10876804 0.07363014 0.01952876 0.05438596
 0.14268243 0.18234443 0.06130521 0.08989205 0.0519943  0.0959596
 0.28652751 0.03171857 0.03258656 0.31515152 0.0675413  0.05351171
 0.05038488 0.20692718 0.04291045 0.18817204 0.11889492 0.04736842
 0.06490385 0.10381078 0.0754717  0.1086262  0.07772021 0.02838063
 0.04365482 0.15294118 0.12522686 0.18483412 0.10909091 0.04093567
 0.05855856 0.07798165 0.11290323 0.25833333 0.06862745 0.14035088
 0.05737705 0.33333333]


# Simulation

In [ ]:
def simulate_budget_roi(df, budget_total, attribution, verbose=False):
    # convert attribution weights to budgets
    budgets = np.ceil(attribution * (budget_total / np.sum(attribution)))  
    
    blacklist = set()
    conversions = set()
    for i in range(df.shape[0]):                                  # simulation loop
        campaign_id = get_campaign_id(df.loc[i]['campaigns']) 
        jid = df.loc[i]['jid']
        if jid not in blacklist:
            if budgets[campaign_id] >= 1:
                budgets[campaign_id] = budgets[campaign_id] - 1
                if(df.loc[i]['conversion'] == 1):
                    conversions.add(jid)
            else:
                blacklist.add(jid)
            
    return len(conversions.difference(blacklist))


def get_campaign_id(x_journey_step):
    return np.argmax(x_journey_step[0:n_campaigns])

In [ ]:
pitches = [0.5, 1.0, 1.5, 2.0, 2.5]
attributions = [lta, fta, uta, tdta]

for i, pitch in enumerate(pitches):
    for j, attribution in enumerate(attributions):
        reward = simulate_budget_roi(df6, 10000, attribution**pitch)
        print('{} {} : {}'.format(pitch, j, reward))

# Main

In [53]:
from bokeh.models.widgets import Panel, Tabs
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.layouts import layout
from bokeh.plotting import curdoc
import bokeh.io
from bokeh.layouts import gridplot
from bokeh.layouts import column
from bokeh.models.widgets import Button, RadioButtonGroup, Select, Slider
from bokeh.models import HoverTool

#output_file
output_file("Attribution_Modelling.html")

n_campaigns = 400

#First Touch Attribution
campaign_idx = range(0,50)
y=fta[campaign_idx] 
x=range(len(fta[campaign_idx]))
p1 = figure(plot_height=500, title="First Touch Attribution",x_axis_label='Campaign ID', 
             x_axis_location='below',
             y_axis_label='Return per impression',
             y_axis_type='linear',
             y_axis_location='left',
             tools="", plot_width = 500)
p1.vbar(x=x, top=y, width=0.1)
p1.xgrid.grid_line_color = None
p1.y_range.start = 0
#Add a hover tool referring to the formatted columns
hover = HoverTool(tooltips = [('Campaign_Id', '@x'),
                             ('Return per impression', '$y')])
p1.add_tools(hover)
#tab3 = Panel(child=p3, title="LTA")


#Linear attribution
x= range(len(la[campaign_idx]))
y=la[campaign_idx]
p2 = figure(plot_height=500, title="Linear Attribution",x_axis_label='Campaign ID', 
             x_axis_location='below',
             y_axis_label='Return per impression',
             y_axis_type='linear',
             y_axis_location='left',
             tools="", plot_width = 500)
p2.vbar(x=x, top=y, width=0.2)
p2.xgrid.grid_line_color = None
p2.y_range.start = 0
# Add a hover tool referring to the formatted columns
hover = HoverTool(tooltips = [('Campaign_Id', '@x'),
                             ('Return per impression', '$y')])
p2.add_tools(hover)


#U shaped attribution
x= range(len(uta[campaign_idx]))
y=uta[campaign_idx]
p31 = figure(plot_height=500, title="U shaped Attribution",x_axis_label='Campaign ID', 
             x_axis_location='below',
             y_axis_label='Return per impression',
             y_axis_type='linear',
             y_axis_location='left',
             tools="", plot_width = 500)
p31.vbar(x=x, top=y, width=0.2)
p31.xgrid.grid_line_color = None
p31.y_range.start = 0
# Add a hover tool referring to the formatted columns
hover = HoverTool(tooltips = [('Campaign_Id', '@x'),
                             ('Return per impression', '$y')])
p31.add_tools(hover)


x= leaderboard['Timestamp']
y= leaderboard['Wightage'] 
p3 = figure(plot_height=500, title="U shaped Attribution",x_axis_label='Campaign ID', 
             x_axis_location='below',
             y_axis_label='Weightage',
             y_axis_type='linear',
             y_axis_location='left',
             tools="", plot_width = 500)
p3.vbar(x=x, top=y, width=0.2)
p3.xgrid.grid_line_color = None
p3.y_range.start = 0
hover = HoverTool(tooltips = [('Campaign_Id', '@x'),
                             ('Weightage', '$y')])
p3.add_tools(hover)





#Time decay attribution

x= range(len(tdta[campaign_idx]))
y=tdta[campaign_idx]
p4 = figure(plot_height=500, title="Time decay Attribution",x_axis_label='Campaign ID', 
             x_axis_location='below',
             y_axis_label='Return per impression',
             y_axis_type='linear',
             y_axis_location='left',
             tools="", plot_width = 500)
p4.vbar(x=x, top=y, width=0.2)
p4.xgrid.grid_line_color = None
p4.y_range.start = 0
# Add a hover tool referring to the formatted columns
hover = HoverTool(tooltips = [('Campaign_Id', '@x'),
                             ('Return per impression', '$y')])
p4.add_tools(hover)


#Last Touch Attribution
y=lta[campaign_idx] 
x=range(len(lta[campaign_idx]))
p6 = figure(plot_height=500, title="Last Touch Attribution",x_axis_label='Campaign ID', 
             x_axis_location='below',
             y_axis_label='Return per impression',
             y_axis_type='linear',
             y_axis_location='left',
             tools="", plot_width = 500)
p6.vbar(x=x, top=y, width=0.1)
p6.xgrid.grid_line_color = None
p6.y_range.start = 0
# Add a hover tool referring to the formatted columns
hover = HoverTool(tooltips = [('Campaign_Id', '@x'),
                             ('Return per impression', '$y')])
p6.add_tools(hover)


#Logistic Regression
# Visualization of the attribution scores
from sklearn.utils.extmath import softmax

keras_logreg = model.get_layer('contributions').get_weights()[0].flatten()[0:n_campaigns]
keras_logreg = softmax([keras_logreg]).flatten()

x= range(len(keras_logreg[campaign_idx]))
y=keras_logreg[campaign_idx]
p7 = figure(plot_height=500, title="Logistic Regression",x_axis_label='Campaign ID', 
             x_axis_location='below',
             y_axis_label='Return per impression',
             y_axis_type='linear',
             y_axis_location='left',
             tools="", plot_width = 500)
p7.vbar(x=x, top=y, width=0.2)
p7.xgrid.grid_line_color = None
p7.y_range.start = 0
# Add a hover tool referring to the formatted columns
hover = HoverTool(tooltips = [('Campaign_Id', '@x'),
                             ('Return per impression', '$y')])
p7.add_tools(hover)




from bokeh.layouts import row
row1 = row(p1)
row2 = row(p2)
row3 = row(p31,p3)
row8 = row(p4)
row4 = row(p6,p7)
row5 = row()

#col = column(row0,row1,row2)
#grid = gridplot([p1,p2,p3,p4,p5,p6], ncols=3)
tab1 = Panel(child = row1, title = 'First Touch Attribution')
tab2 = Panel(child = row2, title = 'Linear Attribution')
tab3 = Panel(child = row3, title = 'U Shaped Attribution')
tab4 = Panel(child = row8, title = 'Time Decay Attribution')
tab5 = Panel(child = row4, title = 'LTA & Logistic Regression')
tab6 = Panel(child = , title = 'Simulation')  
tabs = Tabs(tabs = [tab1, tab2, tab3, tab4, tab5, tab6])

# show the results
show(tabs)
